In [ ]:
# Dependencies
import requests
import json
import pprint
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# API call setup
url = "https://www.alphavantage.co/query"

function = "TIME_SERIES_DAILY"
symbol = ""
api_key = "SMBQT4NW44MXXREK"
output = "full"

data = { "function": function,
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": output} 
page = requests.get(url, params = data)
pprint.pprint(page.json())

In [ ]:
# Make a DataFrame
mydictionary = page.json()

df = pd.DataFrame.from_dict(mydictionary["Time Series (Daily)"], orient='index')

df = df.reset_index()
df = df.sort_index(ascending=False)
df = df.reset_index()
df.head()

In [ ]:
# Convert Day/Month/Year
month = []
day=[]
year = []
for index, row in df.iterrows():
    a = row["index"]
    datee = datetime.strptime(a, "%Y-%m-%d")
    monthdate = datee.strftime("%B")
    daydate = datee.strftime("%A") 
    yeardate = datee.strftime("%Y") 
    month.append(monthdate)
    year.append(yeardate)
    day.append(daydate) 

df["Month"] = month
df["Day"] = day
df["Year"]= year
df["Company"] = symbol
df.head()

In [ ]:
# Calculate Daily change
change = []

counter = 0
while counter < (len(df["index"]) -1) :
    open_num = df["1. open"][counter]
    close_num = df["4. close"][counter]
    calc = (float(close_num) - float(open_num)) / float(open_num)
    change.append(calc)
    counter += 1

change.append(0)
df["daily change"] = change
df.head()

In [ ]:
# Change columns from strings to numeric
df["Year"] = df["Year"].apply(pd.to_numeric)
df["2. high"] = df["2. high"].apply(pd.to_numeric)
df["3. low"] = df["3. low"].apply(pd.to_numeric)
df["5. volume"] = df["5. volume"].apply(pd.to_numeric)
df.head()

In [ ]:
# Delete data outside scope
above_df = df.loc[df["Year"] >= 2007, :]
below_df = above_df.loc[(above_df["Year"] < 2018), :]
master_df = below_df
del master_df["level_0"]
master_df = master_df.reset_index()
master_df.head()

In [ ]:
# Make GroupBy easier
day_to_num = []
month_to_num = []

x = 0

while x < len(master_df["Day"]):
    
    if master_df["Day"][x] == "Monday":
        day_to_num.append(1)
    elif master_df["Day"][x] == "Tuesday":
        day_to_num.append(2)
    elif master_df["Day"][x] == "Wednesday":
        day_to_num.append(3)
    elif master_df["Day"][x] == "Thursday":
        day_to_num.append(4)
    else:
        day_to_num.append(5)
        
    x += 1

z = 0

while z < len(master_df["Month"]):
    
    if master_df["Month"][z] == "January":
        month_to_num.append(1)
    elif master_df["Month"][z] == "February":
        month_to_num.append(2)
    elif master_df["Month"][z] == "March":
        month_to_num.append(3)
    elif master_df["Month"][z] == "April":
        month_to_num.append(4)
    elif master_df["Month"][z] == "May":
        month_to_num.append(5)
    elif master_df["Month"][z] == "June":
        month_to_num.append(6)
    elif master_df["Month"][z] == "July":
        month_to_num.append(7)
    elif master_df["Month"][z] == "August":
        month_to_num.append(8)
    elif master_df["Month"][z] == "September":
        month_to_num.append(9)
    elif master_df["Month"][z] == "October":
        month_to_num.append(10)
    elif master_df["Month"][z] == "November":
        month_to_num.append(11)
    else:
        month_to_num.append(12)
        
    z += 1

master_df["Date to Num"] = day_to_num
master_df["Month to Num"] = month_to_num
master_df.head()

In [ ]:
# Calculate Volatility
Vol = []

x = 0

while x < len(master_df["Month"]):
    new = master_df["2. high"][x]
    old = master_df["3. low"][x]
    change = 0 
    change = (new - old) /old
    Vol.append(change)
    x += 1

master_df["Volatility"] = Vol
master_df.head()

In [ ]:
# Make a grouped data frame for days
day_change = master_df.groupby(["Date to Num"])

MeanofChange = (day_change["daily change"].mean()) * 100
SumofVolu = day_change["5. volume"].sum()
MeanofVol = (day_change["Volatility"].mean()) * 100

DayGrouped = pd.DataFrame({"Daily Change Average" : MeanofChange,
                       "Total Volume" : SumofVolu,
                          "Volatility": MeanofVol})

DayGrouped.head()

In [ ]:
# Make a grouped data frame for months
month_change = master_df.groupby(["Month to Num"])

mMeanofChange = (month_change["daily change"].mean()) * 100
mSumofVolu = month_change["5. volume"].sum()
mMeanofVol = (month_change["Volatility"].mean()) * 100

MonthGrouped = pd.DataFrame({"Monthly Change Average" : mMeanofChange,
                       "Total Volume" : mSumofVolu,
                            "Volatility": mMeanofVol})

MonthGrouped.head(12)

In [ ]:
# Days/Months for plot labels
Days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
Months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

In [ ]:
# Invert DF to have oldest date at start
companies = master_df["Company"].unique()
comp_zero = master_df.loc[master_df["Company"] == companies[0], :]

invert_df = comp_zero.sort_index(ascending=False)

# Find index number of year breaks to plot on line chart
year_break = [0]
counter = 0
z = 0
while z < (len(invert_df["Year"]) - 2):
    if invert_df["Year"][z] == invert_df["Year"][(z+1)]:
        counter += 1
    else:
        year_break.append(counter)
        counter += 1
    
    z += 1
    
year_break

In [ ]:
# Plot closing price to see trend
x_axis = np.arange(len(invert_df["Company"]))
plt.figure(figsize=(25,10))

f = 0
year_label = invert_df["Year"].unique()
year_count = 0
for year in year_break: 
    plt.axvline(x=(year), linestyle = "--", linewidth = 1, color = "black", alpha = .25, gid = year_label[year_count])
    f += 1
    year_count += 1

plt.xticks(year_break, invert_df["Year"].unique())
plt.yticks([],[])
plt.ylabel("Closing Price Trend")
plt.title(symbol + " Daily Price Change")

data = (invert_df["4. close"])

plt.plot(x_axis, data)

plt.savefig("Figures/" + symbol + "ClosingPrice")
plt.show()

In [ ]:
# Daily Change Average Chart
x_axis = np.arange(len(Days))
y_axis = DayGrouped["Daily Change Average"]

plt.figure(figsize=(10,8))

colors = []
for check in DayGrouped["Daily Change Average"]:
    if check >= 0:
        color = "black"
        colors.append(color)
    else:
        color = "r"
        colors.append(color)

plt.bar(x_axis, y_axis, color = colors, alpha=0.8, align="edge")

plt.axhline(y=0, linestyle='-', linewidth = 1, color = "black", alpha = .6)

tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, Days)

plt.ylabel("Average of Open/Close Change (%)")
plt.title(symbol + " Daily Price Change")

plt.savefig("Figures/" + symbol + "DDCA")
plt.show()

In [ ]:
# Monthly Change Average Chart
x_axis = np.arange(len(Months))
y_axis = MonthGrouped["Monthly Change Average"]

plt.figure(figsize=(10,8))

mcolors = []

for z in MonthGrouped["Monthly Change Average"]:
    if z >= 0:
        color = "black"
        mcolors.append(color)
    else:
        color = "red"
        mcolors.append(color)

plt.bar(x_axis, y_axis, color=mcolors, alpha=0.8, align="edge")

plt.axhline(y=0, linestyle='-', linewidth = 1, color = "black", alpha = .6)

tick_locations = [value+.4 for value in x_axis]
plt.xticks(tick_locations, Months)

plt.ylabel("Average of Open/Close Price (%)")
plt.title(symbol + " Monthly Price Change")

string = symbol + "MDCA"

plt.savefig("Figures/" + symbol + "MDCA")
plt.show()

In [ ]:
# Daily Volatility
x_axis = np.arange(len(Days))
y_axis = DayGrouped["Volatility"]

plt.figure(figsize=(10,8))

plt.bar(x_axis, y_axis, color='black', alpha=0.8, align="edge")

tick_locations = [value+0.4 for value in x_axis]
plt.xticks(tick_locations, Days)

maxi = DayGrouped["Volatility"].max()
mini = DayGrouped["Volatility"].min()
higher = (maxi + (maxi/20))
lower = (mini - (mini/10))

plt.ylim(lower,higher)

plt.ylabel("High/Low Volatility (%)")
plt.title(symbol + " Daily Volatility")

plt.savefig("Figures/" + symbol + "DV")
plt.show()

In [ ]:
# Monthly Volatility
x_axis = np.arange(len(Months))
y_axis = MonthGrouped["Volatility"]

plt.figure(figsize=(10,8))

plt.bar(x_axis, y_axis, color='black', alpha=0.8, align="edge")

plt.axhline(y=0, linestyle='-', linewidth = 1, color = "black", alpha = .6)

tick_locations = [value+.4 for value in x_axis]
plt.xticks(tick_locations, Months)

maxi = MonthGrouped["Volatility"].max()
mini = MonthGrouped["Volatility"].min()
higher = (maxi + (maxi/10))
lower = (mini - (mini/10))

plt.ylim(lower,higher)

plt.ylabel("High/Low Volatility (%)")
plt.title(symbol + " Monthly Volatility")

plt.savefig("Figures/" + symbol + "MV")
plt.show()

In [ ]:
x_axis = np.arange(len(invert_df["Company"]))
plt.figure(figsize=(25,10))

f = 0
year_label = invert_df["Year"].unique()
year_count = 0
for year in year_break: 
    plt.axvline(x=(year), linestyle = "--", linewidth = 1, color = "black", alpha = .25, gid = year_label[year_count])
    f += 1
    year_count += 1

plt.xticks(year_break, invert_df["Year"].unique())
plt.axhline(y=0, linestyle='--', linewidth = 2, color = "black", alpha = .4)
plt.ylabel("Daily Open/Close Change (%)")
plt.title(symbol + " Daily Price Change")

data = (invert_df["daily change"] * 100)

plt.plot(x_axis, data)

plt.savefig("Figures/" + symbol + "VolLineFull")
plt.show()